In [1]:
import pandas as pd
import googlemaps
import os
import json
from dotenv import load_dotenv

load_dotenv()

GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
gmaps = googlemaps.Client(key=GMAPS_API_KEY)

In [2]:
filename = "../../raw_data/sfo/listings.csv"
# filename = "sample/listings.csv"
df = pd.read_csv(filename)

df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,NaN,Western Addition,37.769310,-122.433860,Entire home/apt,132,2,257,2020-07-30,1.91,1,177
1,5858,Creative Sanctuary,8904,Philip And Tania,NaN,Bernal Heights,37.745110,-122.421020,Entire home/apt,235,30,111,2017-08-06,0.81,1,365
2,7918,A Friendly Room - UCSF/USF - San Francisco,21994,Aaron,NaN,Haight Ashbury,37.765550,-122.452130,Private room,56,32,19,2020-03-06,0.14,9,365
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,NaN,Haight Ashbury,37.765550,-122.452130,Private room,56,32,8,2018-09-12,0.11,9,365
4,8339,Historic Alamo Square Victorian,24215,Rosy,NaN,Western Addition,37.775250,-122.436370,Entire home/apt,750,5,28,2019-06-28,0.21,2,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,44758668,Open Mission District Studio in SF!,4430421,Landmark,NaN,Mission,37.765850,-122.418580,Entire home/apt,71,30,0,NaN,NaN,132,180
7270,44799193,Cozy Downtown Studio Across from Hilton SF,4430421,Landmark,NaN,Downtown/Civic Center,37.784310,-122.411900,Entire home/apt,62,30,0,NaN,NaN,132,173
7271,44805385,Newly Remodel Luxury Parkmerced townhouse,339209014,Joe,NaN,Lakeshore,37.718910,-122.483230,Entire home/apt,399,1,0,NaN,NaN,1,364
7272,44810526,Spacious private bd/ba in a 2bd/2ba in Mission...,19487100,Ralph,NaN,South of Market,37.777430,-122.394470,Private room,120,120,0,NaN,NaN,1,173


In [3]:
def get_places_nearby(lat,lng):
    return gmaps.places_nearby(
                    location=(lat, lng),
                    keyword="",
                    language="en-SG",
                    rank_by="distance",
                    type=["transit_station", "point_of_interest"],
                )

def find_type_in_results(response, name_type="transit_station"):
    for i in range(len(response["results"])):
        if name_type in response["results"][i]["types"]:
            cord = response["results"][i]["geometry"]["location"]
            return (cord["lat"], cord["lng"])
            break
            
def calculate_distance(origin, destination):
    #e.g (37.769348,-122.433699)
    return gmaps.distance_matrix(origin, destination)

In [ ]:
for x in df.index: 
    listing_id = int(df["id"][x])
    origin = (df["latitude"][x], df["longitude"][x])
    
    response = get_places_nearby(origin[0], origin[1])

    destination = find_type_in_results(response)

    distance = calculate_distance(origin, destination)
    
    merged = {"listing_id":listing_id, **response, **distance}
    
    with open('raw.txt', 'a') as file:
        file.write(json.dumps(merged))
    